## Matrix and Vocabulary Construction

In [251]:
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps

In [252]:
news = pd.read_csv("./estadao_noticias_eleicao.csv", encoding="utf-8")

In [253]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")

In [254]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [255]:
tokens_lists = content.apply(lambda text: text.lower().split())

In [256]:
tokens = [token for tokens_list in tokens_lists for token in tokens_list]

In [257]:
matrix, vocab = co_occurrence_matrix(tokens)

## Consult Bigram Frequency

In [258]:
consultable_matrix = matrix.tocsr()

In [259]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

### Example

In [260]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1, w2)

3

## Top-3 em ordem decrescente

In [301]:
def top3(w1):
    top3 = {}
    for w in consultable_matrix[vocab[w1]].indices:
        if(len(top3) < 3):
            top3[w] = consultable_matrix[vocab[w1], w]
        else:
            if(consultable_matrix[vocab[w1], w] > consultable_matrix[vocab[w1], min(top3, key=top3.get)]):
                del top3[min(top3, key=top3.get)]
                top3[w] = consultable_matrix[vocab[w1], w]
                
    words = [str(list(vocab.keys())[list(vocab.values()).index(list(top3.keys())[0])]), str(list(vocab.keys())[list(vocab.values()).index(list(top3.keys())[1])]), str(list(vocab.keys())[list(vocab.values()).index(list(top3.keys())[2])])]
    return words
                

In [302]:
print(top3("presidente"))

['dilma', 'da', 'do']


## Expandir consulta original

In [303]:
def getIDs(search):
    searchWords = search.split(" ")
    wordIDs = []
    if(len(searchWords) >=2):
        booleanOperator = searchWords[1]
        wordIDs1 = []
        wordIDs2 = []
        for i in tokens_lists:
            for w in tokens_lists[i]:
                if(searchWords[0].lower() == w.lower()):
                    wordIDs1.append(news.values[i][2])
                if(searchWords[2].lower() == w.lower()):
                    wordIDs2.append(news.values[i][2])
                if((searchWords[1] in wordIDs1) and searchWords[2] in wordIDs2):
                    break
        if(booleanOperator == "AND"):
            wordIDs = set(wordIDs1).intersection(wordIDs2)
        else:
            wordIDs = set(wordIDs1).union(wordIDs2)
    else:
        for i in range(0, len(tokens_lists)):
            for w in range(0, len(token_lists)):
                if(searchWords[0].lower() == token_lists[w].lower()):
                    wordIDs.append(news.values[i][2])  
                    break
    return wordIDs


In [304]:
def expandQuery(word, top3):
    for w in top3:
        query = word + " OR " + w
        print(query)
        print(getIDs(query))

In [305]:
top3 = top3("dilma")
print(top3)
expandQuery("dilma", top3)

['e', 'rousseff', 'rousseff,']
dilma OR e


KeyError: "None of [['pt', 'espera', '30', 'mil', 'pessoas', 'em', 'festa', 'na', 'esplanada', 'objetivo', 'é', 'demonstrar', 'apoio', 'popular', 'a', 'dilma', 'e', 'tentar', 'reduzir', 'questionamentos', 'dos', 'adversários', 'brasília', '-', 'após', 'o', 'desgaste', 'provocado', 'com', 'o', 'lançamento', 'de', 'medidas', 'impopulares,', 'o', 'anúncio', 'de', 'um', 'ministério', 'com', 'nomes', 'contestados', 'pelos', 'movimentos', 'sociais', 'e', 'o', 'surgimento', 'de', 'manifestações', 'pró-impeachment,', 'o', 'pt', 'pretende', 'mobilizar', 'pelo', 'menos', '30', 'mil', 'pessoas', 'para', 'encher', 'a', 'esplanada', 'dos', 'ministérios', 'e', 'tornar', 'a', 'posse', 'da', 'presidente', 'dilma', 'rousseff', 'um', 'grande', 'endosso', 'popular', 'ao', 'segundo', 'mandato', 'da', 'petista.', 'no', 'palácio', 'do', 'planalto', 'a', 'avaliação', 'é', 'de', 'que', 'a', 'mobilização', 'da', 'sociedade', 'civil', 'é', 'fundamental', 'para', 'mostrar', 'que', 'a', 'presidente', 'possui', 'l